In [1]:
import numpy as np
import pickle
import os

In [2]:
def create_directory(path):
    if not os.path.exists(path):
        print('Creating directory: ', path)
        os.makedirs(path)

In [3]:
def process_data(lines, offset, annots):
    #print(annots)    
    tokens = lines[0].split(",")
    if len(tokens)==1:
        lines = lines[1:]
    
    data = []
    for line in lines:
        tokens = line.split(",")
        sensor_id = int(tokens[1])
        if sensor_id in [1, 4, 9, 11]:
            t = int(tokens[0])
            x = float(tokens[3])
            y = float(tokens[4])
            z = float(tokens[5])
            data.append([t, sensor_id, x, y, z])
        
    data = np.array(data)  
    data[:, 0] = data[:, 0] - data[0, 0]
    data[:, 0] = data[:, 0]/1e9
        
    if len(annots)==0:                               
        res_annots = np.array([])        
    else:
        annots.sort(axis=0)
        annots = annots[annots[:, 1]<1000, :]
        annots[:, 0] = annots[:, 0] - offset
                    
    return data, annots

In [4]:
dest_folder = 'C:/ASM/DevData/eating/datasets'
create_directory(dest_folder)

In [5]:
subject_serial = 0
all_data = []

In [6]:
#read data usc
srcFolder = 'D:/Box Sync/MyData/Eating m2fed/usc_meals';
subjects = [
    ['0102', 94.6],
    ['0103', 67.5],
    ['0301', 59.8],
    ['0601', 26.1],
    ['0602', 16.0],
    ['0603', 17.1],
    ['0801', 82.0],
    ['0803', 67.9],
    ['0901', 63.9],
    ['0902', 123.2],
    ['1001', 89.5],    
    ['1303', 123.8],
    ['1304', 107.1],
    ['1305', 135.5]]


for subj in range(len(subjects)):
    subject_id = subjects[subj][0]
    print(subject_id)
    file_path = srcFolder+'/sensor_data/'+subject_id+'_right.wada'
    with open(file_path) as file:
        raw_data = file.readlines()

    file_path = srcFolder+'/annotations/processed/'+ subject_id+ '_annotations_right.csv'
    annots = np.genfromtxt(file_path, delimiter=",")
    
    offset = subjects[subj][1]

    data, annots = process_data(raw_data, offset, annots)
    if subj ==0:             
        data = data[data[:,0]<42*60, :]        
    
    #np.savetxt(dest_file_path+'data_'+str(file_serial), data, delimiter=",")
    #np.savetxt(dest_file_path+'annots_'+str(file_serial), annots, delimiter=",")
    
    d = {'data':data, 'annots':annots, 'type':'usc'}
    all_data.append([d])    
   

0102
0103
0301
0601
0602
0603
0801
0803
0901
0902
1001
1303
1304
1305


In [7]:
#read data uva
session_counts = [2, 4, 2, 2, 4, 5, 1]
srcFolder = 'D:/Box Sync/MyData/Eating m2fed/uva_meals'

file_path = srcFolder +'/offsets.csv'
all_offsets = []
with open(file_path) as file:
    lines = file.readlines()
    for line in lines:       
        tokens = line.split(",")
        ofs = []
        for t in tokens:
            t = t.rstrip()            
            if len(t)>0:
                t= float(t)
                ofs.append(t)
        all_offsets.append(ofs)
        
print(all_offsets)

for subj in range(7):
    sess_count = session_counts[subj]
    subject_data = []
    for sess in range(sess_count):
        print('Processing ', (subj+1), (sess+1))
        file_path = srcFolder+'/subject_'+ str(subj+1)+ '/subject'+ str(subj+1)+ '_right_session'+ str(sess+1)+'.wada'
        with open(file_path) as file:
            raw_data = file.readlines()
        
        file_path = srcFolder + '/annotations/processed/subject'+ str(subj+1)+ '_annotations_right_session'+ str(sess+1) + '.csv'
        annots = np.genfromtxt(file_path, delimiter=",")
        
        offset = all_offsets[subj][sess]
        
        data, annots = process_data(raw_data, offset, annots)
        #np.savetxt(dest_file_path+'data_'+str(file_serial), data, delimiter=",")
        #np.savetxt(dest_file_path+'annots_'+str(file_serial), annots, delimiter=",")
        d = {'data':data, 'annots':annots, 'type':'uva'}
        subject_data.append(d)
        
    all_data.append(subject_data)
        

[[16.2, 7.8], [10.5, 18.3, 8.3, 23.3, 149.5], [9.5, 21.1], [7.9, 16.0], [8.0, 5.6, 4.6, 8.5], [8.1, 30.3, 3.4, 15.6, 5.5], [7.4]]
Processing  1 1
Processing  1 2
Processing  2 1
Processing  2 2
Processing  2 3
Processing  2 4
Processing  3 1
Processing  3 2
Processing  4 1
Processing  4 2
Processing  5 1
Processing  5 2
Processing  5 3
Processing  5 4
Processing  6 1
Processing  6 2
Processing  6 3
Processing  6 4
Processing  6 5
Processing  7 1


In [8]:
with open(dest_folder+'/our_lab_dataset.pkl', 'wb') as file:
    pickle.dump(all_data, file)

In [9]:
print("total subjects in lab: ", len(all_data))
for s in range(len(all_data)):
    print('Subject {}: Session count {}'.format(s, len(all_data[s])))
    

total subjects in lab:  21
Subject 0: Session count 1
Subject 1: Session count 1
Subject 2: Session count 1
Subject 3: Session count 1
Subject 4: Session count 1
Subject 5: Session count 1
Subject 6: Session count 1
Subject 7: Session count 1
Subject 8: Session count 1
Subject 9: Session count 1
Subject 10: Session count 1
Subject 11: Session count 1
Subject 12: Session count 1
Subject 13: Session count 1
Subject 14: Session count 2
Subject 15: Session count 4
Subject 16: Session count 2
Subject 17: Session count 2
Subject 18: Session count 4
Subject 19: Session count 5
Subject 20: Session count 1


In [10]:
srcFolder = 'D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/'
subjects = ['abu','emi','sarah','meiyi']

all_data = []
for subj in subjects:    
    folder_path = srcFolder + subj + '_home_noneat'
    files = os.listdir(folder_path);
    for f in files:
        file_path = folder_path+ '/' + f
        print(file_path)
        with open(file_path) as file:
            raw_data = file.readlines()
        
        annots = [];        
        offset = 0;        
       
        data, annots = process_data(raw_data, offset, annots)
        #np.savetxt(dest_file_path+'accel_'+str(file_serial), accel, delimiter=",")
        d = {'data':data, 'annots':annots, 'subject_name':subj}
        all_data.append(d)        

D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/abu_home_noneat/14402D24F306CE8-m2fed-04-01-Right-2017-03-08-22-39-00.wada
D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/abu_home_noneat/14402D24F306CE8-m2fed-04-01-Right-2017-03-08-23-30-58.wada
D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/abu_home_noneat/14452D1EF8D6D9E-m2fed-04-01-Right-2013-03-29-06-08-27.wada
D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/emi_home_noneat/14422D2DF3B6AA2-m2fed-03-03-Right-2013-02-06-02-28-46.wada
D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/emi_home_noneat/14422D2DF3B6AA2-m2fed-03-03-Right-2013-02-07-06-05-48.wada
D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/emi_home_noneat/14422D2DF3B6AA2-m2fed-03-03-Right-2013-02-12-01-21-54.wada
D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/emi_home_noneat/14422D2DF3B6AA2-m2fed-03-03-Right-2013-02-12-05-46-34.wada
D:/Box Sync/MyData/Eating m2fed/uva_noneat_home/sarah_home_noneat/14402D24F306CE8-m2fed-01-01-Right-2017-02-27-16-29-48.wada
D:/Box Sync/My

In [11]:
with open(dest_folder+'/our_free_dataset.pkl', 'wb') as file:
    pickle.dump(all_data, file)